In [1]:
import pandas as pd
import dask.dataframe as dd
import json

In [2]:
charFile = 'Characteristics_Final.csv'
imagesFile = 'Images_Final.csv'
reviewsFile = 'reviews.csv'
char = dd.read_csv(charFile, blocksize=700e6)
images = dd.read_csv(imagesFile, blocksize=700e6)
reviews = dd.read_csv(reviewsFile, blocksize=700e6)

In [3]:
char.head(2)

,review_id,result
0,1,"[{""name"": ""Fit"", ""characteristic_id"": 1, ""valu..."
1,2,"[{""name"": ""Fit"", ""characteristic_id"": 1, ""valu..."


In [4]:
char = char.rename(columns={"result": "char"})

In [5]:
images = images.rename(columns={"url": "photos"})


In [6]:
images.head(2)

,review_id,photos
0,5,"[""https://images.unsplash.com/photo-1560570803..."
1,9,"[""https://images.unsplash.com/photo-1542574621..."


In [7]:
reviews.head(2)

,id,product_id,rating,date,summary,body,recommend,reported,reviewer_name,reviewer_email,response,helpfulness
0,1,1,5,1596080481467,This product was great!,I really did or did not like this product base...,True,False,funtime,first.last@gmail.com,<NA>,8
1,2,1,4,1610178433963,This product was ok!,I really did not like this product solely beca...,False,False,mymainstreammother,first.last@gmail.com,<NA>,2


In [8]:
reviewsMergedWithPhotos = dd.merge(reviews, images, left_on='id', right_on='review_id', how='left')

In [9]:
reviewsMergedWithPhotosAndChar = dd.merge(reviewsMergedWithPhotos, char, left_on='id', right_on='review_id', how='left')

In [10]:
reviewsMergedWithPhotosAndChar = reviewsMergedWithPhotosAndChar.drop(['review_id_x','review_id_y'], axis=1)


In [11]:
reviewsMergedWithPhotosAndChar

,id,product_id,rating,date,summary,body,recommend,reported,reviewer_name,reviewer_email,response,helpfulness,photos,char
npartitions=2,,,,,,,,,,,,,,
,int64,int64,int64,int64,string,string,bool,bool,string,string,string,int64,string,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [12]:
empty = json.dumps([])
reviewsMergedWithPhotosAndChar = reviewsMergedWithPhotosAndChar.fillna(value={"photos":empty})

In [13]:
reviewsMergedWithPhotosAndChar.compute()
reviewsMergedWithPhotosAndChar.to_csv('FINAL.csv', index=False)

['/Users/Chris/hackreactor/Python/Final/FINAL.csv/0.part',
 '/Users/Chris/hackreactor/Python/Final/FINAL.csv/1.part']